In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score

# File Path 
data = pd.read_excel(r"INSERT HERE")

# columns
columns_of_interest = [f"X{i}" for i in range(1, 28)] + ["status", "industry"]
model_data = data[columns_of_interest]

# Prod & Service
production_industries = ['A', 'C', 'F']
service_industries = ['G', 'H', 'I', 'J', 'L', 'M', 'N', 'P', 'Q', 'R', 'S']

# Splitting Prod
production_data = model_data[model_data['industry'].isin(production_industries)]
X_prod = production_data.drop(['status', 'industry'], axis=1)
y_prod = production_data['status']
X_train_prod, X_test_prod, y_train_prod, y_test_prod = train_test_split(X_prod, y_prod, test_size=0.30, random_state=99, stratify=y_prod)

# Splitting Service
service_data = model_data[model_data['industry'].isin(service_industries)]
X_serv = service_data.drop(['status', 'industry'], axis=1)
y_serv = service_data['status']
X_train_serv, X_test_serv, y_train_serv, y_test_serv = train_test_split(X_serv, y_serv, test_size=0.30, random_state=99, stratify=y_serv)

# RF
rf_model_prod = RandomForestClassifier(n_estimators=300, max_depth=None, max_features='log2', min_samples_leaf=2, min_samples_split=5, random_state=99)
rf_model_serv = RandomForestClassifier(n_estimators=300, max_depth=None, max_features='log2', min_samples_leaf=2, min_samples_split=5, random_state=99)
rf_model_prod.fit(X_train_prod, y_train_prod)
rf_model_serv.fit(X_train_serv, y_train_serv)

# Predicting for each industry
industries = model_data['industry'].unique()
for industry in industries:
    industry_data = model_data[model_data['industry'] == industry]
    X_industry = industry_data.drop(['status', 'industry'], axis=1)
    y_industry = industry_data['status']
    X_train_industry, X_test_industry, y_train_industry, y_test_industry = train_test_split(X_industry, y_industry, test_size=0.30, random_state=99, stratify=y_industry)

    if industry in production_industries:
        model = rf_model_prod
    else:
        model = rf_model_serv

    y_pred = model.predict(X_test_industry)
    y_proba = model.predict_proba(X_test_industry)[:, 1]

    # Results 
    accuracy = accuracy_score(y_test_industry, y_pred)
    conf_matrix = confusion_matrix(y_test_industry, y_pred)
    class_report = classification_report(y_test_industry, y_pred)
    auc_score = roc_auc_score(y_test_industry, y_proba)

    print(f"Results for industry {industry}:")
    print("Accuracy:", accuracy)
    print("AUC Score:", auc_score)